import packages

In [1]:
#
import tensorflow as tf
import numpy as np

Reset default graph

In [2]:
tf.reset_default_graph()

Load mnist data from datasets/mnist. 

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('datasets/mnist')

Extracting datasets/mnist/train-images-idx3-ubyte.gz
Extracting datasets/mnist/train-labels-idx1-ubyte.gz
Extracting datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting datasets/mnist/t10k-labels-idx1-ubyte.gz


Try the minist.train.next_batch function to observe the data

In [5]:
X_batch, y_batch = mnist.train.next_batch(50)

In [6]:
from sklearn.datasets import fetch_mldata

Construction phase of DNN:
1. Input layer
2. Hidden layer 1 of 300 neurons, using ReLU activation function
3. Hidden layer 2 of 100 neurons, using ReLU activation function
4. Output layer

This DNN applies gradient descent to minimize the entropy loss function.
This DNN has an "accuracy" node to calculate the accuracy

In [7]:
n_input = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

In [8]:
X = tf.placeholder(tf.float32, shape=(None, n_input), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

In [9]:
with tf.name_scope('DNN'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name='hidden1')
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits  = tf.layers.dense(hidden2, n_output, activation=None, name='logits')

In [10]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits), name='loss')

In [11]:
with tf.name_scope('training'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss, name='training_op')

In [12]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    accuracy_summary = tf.summary.scalar('Accuracy', accuracy)

Execution phase of DNN:
1. Run 50 epoches at a batch size 100
2. Print training and validation accuracy after each epoch, also save them to log file
3. Save the final model at tmp/mnist_dnn_final.ckpt

In [13]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer_training = tf.summary.FileWriter('logs/mnist_dnn/train', graph=tf.get_default_graph())
file_writer_val     = tf.summary.FileWriter('logs/mnist_dnn/val')

In [14]:
n_epoch = 50
batch_size = 100
n_data = mnist.train.num_examples
n_step = int(np.ceil(n_data/batch_size))

In [15]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for step in range(n_step):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
        (train_accuracy_val, train_accuracy_sum) = \
            sess.run([accuracy, accuracy_summary], feed_dict={X: mnist.train.images, y: mnist.train.labels})
        
        (val_accuracy_val, val_accuracy_sum) = \
            sess.run([accuracy, accuracy_summary], feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        
        print("Epoch: {}, training accuray: {:.4f}, validation accuracy: {:.4f}".format(epoch, train_accuracy_val, val_accuracy_val))
        
        global_step = n_step * epoch + step
        file_writer_training.add_summary(train_accuracy_sum, global_step=global_step)
        file_writer_val.add_summary(val_accuracy_sum, global_step=global_step)
    saver.save(sess, 'tmp/mnist_dnn_final.ckpt')
    file_writer_training.close()
    file_writer_val.close()

Epoch: 0, training accuray: 0.5484, validation accuracy: 0.5588
Epoch: 1, training accuray: 0.6894, validation accuracy: 0.6938
Epoch: 2, training accuray: 0.7623, validation accuracy: 0.7632
Epoch: 3, training accuray: 0.8025, validation accuracy: 0.8056
Epoch: 4, training accuray: 0.8229, validation accuracy: 0.8292
Epoch: 5, training accuray: 0.8367, validation accuracy: 0.8438
Epoch: 6, training accuray: 0.8473, validation accuracy: 0.8574
Epoch: 7, training accuray: 0.8558, validation accuracy: 0.8628
Epoch: 8, training accuray: 0.8627, validation accuracy: 0.8686
Epoch: 9, training accuray: 0.8690, validation accuracy: 0.8760
Epoch: 10, training accuray: 0.8735, validation accuracy: 0.8800
Epoch: 11, training accuray: 0.8774, validation accuracy: 0.8846
Epoch: 12, training accuray: 0.8807, validation accuracy: 0.8878
Epoch: 13, training accuray: 0.8836, validation accuracy: 0.8908
Epoch: 14, training accuray: 0.8865, validation accuracy: 0.8928
Epoch: 15, training accuray: 0.8889

Visualize the DNN and the learning curve in TensorBoard